In [6]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg, select_category_value_agg, cnt_encoding
from mv_wg_avg import exp_weight_avg

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals, impute_avg
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
b_id = 'SK_ID'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'is_train', 'is_test', 'SK_ID_PREV']

2018-08-07 22:40:12,677 logger 19 [INFO]    [logger_func] start 
2018-08-07 22:40:12,722 logger 19 [INFO]    [logger_func] start 
2018-08-07 22:40:12,722 logger 19 [INFO]    [logger_func] start 


In [26]:
# base = pd.read_csv('../data/base.csv')[unique_id].to_frame()
app = pd.read_csv('../data/application_train_test_after.csv')
# ccb = pd.read_csv('../data/credit_card_balance.csv')

In [8]:
# ccb['NAME_CONTRACT_STATUS'].drop_duplicates()
# ccb.sort_values(by=[unique_id, 'MONTHS_BALANCE'], ascending=False, inplace=True)
# ccb['NAME_CONTRACT_STATUS'].value_counts()
ccb.head(30)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.500,0.0,877.500,1700.325,1800.000,1800.000,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0.0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,0.0,0.000,2250.000,2250.000,2250.000,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0.0
2,1740877,371185,-7,31815.225,450000,0.0,0.000,0.0,0.000,2250.000,2250.000,2250.000,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0.0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.000,0.0,0.000,11795.760,11925.000,11925.000,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0.0
4,1891521,126868,-1,453919.455,450000,0.0,11547.000,0.0,11547.000,22924.890,27000.000,27000.000,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0.0
5,2646502,380010,-7,82903.815,270000,0.0,0.000,0.0,0.000,4449.105,3825.000,3825.000,80519.040,82773.315,82773.315,0.0,0,0.0,0.0,2.0,Active,7,0.0
6,1079071,171320,-6,353451.645,585000,67500.0,67500.000,0.0,0.000,14684.175,15750.000,15750.000,345433.860,351881.145,351881.145,1.0,1,0.0,0.0,6.0,Active,0,0.0
7,2095912,118650,-7,47962.125,45000,45000.0,45000.000,0.0,0.000,0.000,264.690,0.000,44735.310,47962.125,47962.125,1.0,1,0.0,0.0,51.0,Active,0,0.0
8,2181852,367360,-4,291543.075,292500,90000.0,289339.425,0.0,199339.425,130.500,4093.515,4093.515,285376.410,286831.575,286831.575,3.0,8,0.0,5.0,3.0,Active,0,0.0
9,1235299,203885,-5,201261.195,225000,76500.0,111026.700,0.0,34526.700,6338.340,45000.000,45000.000,192793.275,197224.695,197224.695,3.0,9,0.0,6.0,38.0,Active,0,0.0


In [11]:
num_list = get_numeric_features(data=ccb, ignore=ignore_features)
method_list = ['sum', 'std', 'max', 'min']
amt_list = [col for col in num_list if col.count('AMT')]
amt_draw_list = [col for col in num_list if col.count('AMT_DRAW')]
cnt_list = [col for col in num_list if col.count('CNT')]
cnt_draw_list = [col for col in num_list if col.count('CNT_DRAW')]
month_list = ['MONTHS_BALANCE']

In [21]:
amt_draw_sum = ccb.set_index([unique_id, 'MONTHS_BALANCE'])[amt_draw_list].sum(axis=1).reset_index().rename(columns={0:'AMT_DRAW_sum@'})
cnt_draw_sum = ccb.set_index([unique_id, 'MONTHS_BALANCE'])[cnt_draw_list].sum(axis=1).reset_index().rename(columns={0:'CNT_DRAW_sum@'})

In [27]:
amt_draw_sum = amt_draw_sum.merge(app[[unique_id, 'AMT_INCOME_TOTAL']], on=unique_id, how='inner')
amt_draw_sum['AMT_DRAW_sum@_per_AMT_INCOME_TOTAL'] = amt_draw_sum['AMT_DRAW_sum@'] / amt_draw_sum['AMT_INCOME_TOTAL']

In [31]:
ccb['SK_DPD_diff_MONTHS_BALANCE@'] = ccb['SK_DPD'] - ccb['MONTHS_BALANCE']
ccb['SK_DPD_DEF_diff_MONTHS_BALANCE@'] = ccb['SK_DPD_DEF'] - ccb['MONTHS_BALANCE']
ccb['SK_DPD_diff_SK_DPD_DEF@'] = ccb['SK_DPD'] - ccb['SK_DPD_DEF']

ccb['AMT_TOTAL_RECEIVABLE_diff_AMT_RECIVABLE@'] = ccb['AMT_TOTAL_RECEIVABLE'] - ccb['AMT_RECIVABLE']
ccb['AMT_TOTAL_RECEIVABLE_diff_AMT_RECEIVABLE_PRINCIPAL@'] = ccb['AMT_TOTAL_RECEIVABLE'] - ccb['AMT_RECEIVABLE_PRINCIPAL']
ccb['AMT_RECIVABLE_diff_AMT_RECEIVABLE_PRINCIPAL@'] = ccb['AMT_RECIVABLE'] - ccb['AMT_RECEIVABLE_PRINCIPAL']

ccb['AMT_BALANCE_diff_AMT_RECIVABLE@'] = ccb['AMT_BALANCE'] - ccb['AMT_RECIVABLE']
ccb['AMT_BALANCE_diff_AMT_RECEIVABLE_PRINCIPAL@'] = ccb['AMT_BALANCE'] - ccb['AMT_RECEIVABLE_PRINCIPAL']
ccb['AMT_BALANCE_diff_AMT_TOTAL_RECEIVABLE@'] = ccb['AMT_BALANCE'] - ccb['AMT_TOTAL_RECEIVABLE']

ccb['AMT_DRAWINGS_CURRENT_diff_AMT_DRAWINGS_ATM_CURRENT@'] = ccb['AMT_DRAWINGS_CURRENT'] - ccb['AMT_DRAWINGS_ATM_CURRENT']
ccb['AMT_DRAWINGS_CURRENT_diff_AMT_DRAWINGS_OTHER_CURRENT@'] = ccb['AMT_DRAWINGS_CURRENT'] - ccb['AMT_DRAWINGS_OTHER_CURRENT']
ccb['AMT_DRAWINGS_CURRENT_diff_AMT_DRAWINGS_POS_CURRENT@'] = ccb['AMT_DRAWINGS_CURRENT'] - ccb['AMT_DRAWINGS_POS_CURRENT']

In [33]:
' 1段階の集計 '
prefix = 'ccb_'
df = ccb
level = unique_id
num_list = amt_list
num_list = cnt_list
num_list = month_list
num_list = [col for col in ccb.columns if col.count('_diff_')]
method_list = ['sum', 'std', 'max', 'min']
method_list = ['sum', 'std', 'max']
method_list = ['sum', 'max', 'min']

select_level_agg(base=base, data=df, ignore_list=ignore_features,  level=level, method_list=method_list, num_list=num_list, prefix=prefix)

2018-08-08 08:36:08,063 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_diff_MONTHS_BALANCE@
method: sum
['SK_ID_CURR'] 
2018-08-08 08:36:08,063 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_diff_MONTHS_BALANCE@
method: sum
['SK_ID_CURR'] 
INFO:logger:
num: SK_DPD_diff_MONTHS_BALANCE@
method: sum
['SK_ID_CURR']
../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})
2018-08-08 08:36:09,805 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_diff_MONTHS_BALANCE@_sum@['SK_ID_CURR']'], dtype='object') 
2018-08-08 08:36:09,805 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_diff_MONTHS_BALANCE@_sum@['SK_ID_CURR']'], dtype='object') 
INFO:logger:
base: Index(['SK_ID_CURR'], d

2018-08-08 08:36:10,330 logger 30 [INFO]    [make_npy] 96.0      10009
8.0        4395
9.0        4144
10.0       3950
7.0        3753
11.0       3625
12.0       3447
6.0        3164
13.0       2690
14.0       2498
15.0       2431
16.0       2235
17.0       2094
18.0       1930
5.0        1917
23.0       1571
24.0       1554
4.0        1509
19.0       1495
20.0       1457
22.0       1410
25.0       1389
21.0       1371
26.0       1325
27.0       1185
28.0       1121
3.0        1088
29.0       1006
30.0        926
31.0        861
94.0        813
36.0        777
37.0        761
38.0        759
93.0        744
32.0        741
35.0        732
33.0        727
95.0        723
39.0        708
40.0        706
97.0        701
91.0        692
34.0        686
90.0        684
84.0        678
42.0        676
41.0        670
88.0        668
80.0        667
          ...  
818.0         1
224.0         1
811.0         1
815.0         1
1420.0        1
814.0         1
383.0         1
545.0         1
3

2018-08-08 08:36:10,768 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_diff_MONTHS_BALANCE@
method: sum
['SK_ID_CURR'] 
2018-08-08 08:36:10,768 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_diff_MONTHS_BALANCE@
method: sum
['SK_ID_CURR'] 
INFO:logger:
num: SK_DPD_DEF_diff_MONTHS_BALANCE@
method: sum
['SK_ID_CURR']
2018-08-08 08:36:11,061 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_diff_MONTHS_BALANCE@_sum@['SK_ID_CURR']'], dtype='object') 
2018-08-08 08:36:11,061 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_diff_MONTHS_BALANCE@_sum@['SK_ID_CURR']'], dtype='object') 
INFO:logger:
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_diff_MONTHS_BALANCE@_sum@['SK_ID_CURR']'], dtype='object')
2018-08-08 08:36:11,124 logger 230 [INFO]    [select_level_agg] 
result shape: (356255

2018-08-08 08:36:11,528 logger 30 [INFO]    [make_npy] 96.0     10867
8.0       4477
9.0       4211
10.0      4000
7.0       3816
11.0      3681
12.0      3495
6.0       3183
13.0      2716
14.0      2524
15.0      2442
16.0      2251
17.0      2117
18.0      1945
5.0       1927
23.0      1568
24.0      1553
4.0       1514
19.0      1488
20.0      1458
22.0      1411
25.0      1392
21.0      1372
26.0      1335
27.0      1185
28.0      1137
3.0       1092
29.0      1005
30.0       909
94.0       867
31.0       839
93.0       795
36.0       787
95.0       785
38.0       781
37.0       767
35.0       744
91.0       739
97.0       735
90.0       726
32.0       723
84.0       717
88.0       711
33.0       707
92.0       703
40.0       703
80.0       702
39.0       702
89.0       687
82.0       675
         ...  
65.0        53
61.0        50
59.0        48
63.0        45
62.0        42
58.0        39
114.0       16
119.0       15
120.0       12
118.0       12
122.0       11
110.0       10


2018-08-08 08:36:12,293 logger 30 [INFO]    [make_npy] 0.0        95983
1.0         1741
8.0          535
7.0          351
18.0         325
2.0          313
28.0         221
33.0         212
32.0         183
16.0         168
19.0         119
34.0          89
3.0           87
29.0          85
31.0          83
5.0           79
95.0          66
15.0          65
26.0          62
30.0          58
94.0          54
96.0          43
23.0          42
35.0          41
12.0          37
36.0          32
4.0           31
13.0          31
14.0          26
17.0          22
11.0          20
6.0           19
187.0         19
188.0         19
67.0          18
93.0          18
37.0          18
56.0          18
184.0         17
54.0          16
24.0          14
46.0          13
312.0         13
97.0          13
186.0         12
55.0          12
463.0         11
66.0          11
309.0         10
68.0          10
           ...  
45544.0        1
45496.0        1
22732.0        1
179.0          1
45372.0   

...

2018-08-08 08:36:21,637 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
INFO:logger:
result shape: (356255, 2)
2018-08-08 08:36:21,649 logger 30 [INFO]    [make_npy] 0.000          34428
135000.000      1090
90000.000        979
180000.000       847
45000.000        779
67500.000        671
225000.000       507
112500.000       456
157500.000       318
270000.000       314
202500.000       191
450000.000       190
22500.000        160
360000.000       145
247500.000       125
315000.000       111
27000.000        105
4500.000         104
189000.000       102
9000.000          97
94500.000         97
13500.000         96
121500.000        94
144000.000        93
234000.000        93
139500.000        93
148500.000        90
130500.000        87
175500.000        86
18000.000         83
166500.000        83
72000.000         83
81000.000         82
99000.000         82
108000.000        82
292500.000        80
171000.000        80
36000.000         79
216000.000      

2018-08-08 08:36:22,010 logger 30 [INFO]    [make_npy] 135000.000    4313
90000.000     3930
0.000         3065
180000.000    2970
67500.000     2759
45000.000     2580
112500.000    1978
225000.000    1158
157500.000    1090
270000.000     945
22500.000      523
450000.000     418
54000.000      395
63000.000      378
81000.000      371
72000.000      353
202500.000     351
27000.000      334
108000.000     322
85500.000      316
94500.000      312
49500.000      309
58500.000      295
126000.000     292
99000.000      287
36000.000      282
31500.000      280
405000.000     280
103500.000     275
40500.000      266
13500.000      261
117000.000     249
130500.000     249
76500.000      246
360000.000     240
121500.000     233
315000.000     232
9000.000       202
247500.000     192
175500.000     188
144000.000     179
18000.000      174
4500.000       169
171000.000     168
153000.000     166
139500.000     160
162000.000     156
166500.000     147
148500.000     146
33750.000     

2018-08-08 08:36:22,370 logger 30 [INFO]    [make_npy] 0.000         67415
4500.000        115
2250.000        100
45000.000        86
9000.000         54
13500.000        51
450.000          49
22500.000        46
1350.000         40
900.000          33
6750.000         27
90000.000        24
1800.000         23
135000.000       19
67500.000        17
18000.000        17
2700.000         16
3150.000         15
3600.000         14
225.000          14
11250.000        13
27000.000        12
1575.000         10
45.000            9
225000.000        9
1125.000          8
5400.000          7
8100.000          7
40500.000         7
180000.000        7
5850.000          7
31500.000         7
15750.000         7
4050.000          7
675.000           7
20250.000         7
315.000           6
6300.000          5
441.000           5
540.000           5
270.000           5
567.000           5
7650.000          5
135.000           5
121500.000        5
10350.000         4
36000.000         4
945.0

INFO:logger:0.000          40850
135000.000      1346
90000.000       1239
45000.000       1066
180000.000      1022
67500.000        837
225000.000       670
112500.000       586
157500.000       435
270000.000       426
4500.000         364
22500.000        349
9000.000         303
202500.000       290
13500.000        270
450000.000       232
27000.000        227
360000.000       221
247500.000       212
121500.000       203
18000.000        199
36000.000        197
94500.000        197
315000.000       181
139500.000       181
144000.000       178
99000.000        176
81000.000        175
31500.000        174
49500.000        170
117000.000       170
175500.000       169
40500.000        165
54000.000        164
72000.000        164
76500.000        162
189000.000       162
234000.000       161
108000.000       161
148500.000       159
58500.000        157
130500.000       157
184500.000       156
63000.000        154
166500.000       153
103500.000       153
171000.000       152
1

2018-08-08 08:36:23,174 logger 222 [INFO]    [select_level_agg] 
num: AMT_DRAWINGS_CURRENT_diff_AMT_DRAWINGS_POS_CURRENT@
method: min
['SK_ID_CURR'] 
2018-08-08 08:36:23,174 logger 222 [INFO]    [select_level_agg] 
num: AMT_DRAWINGS_CURRENT_diff_AMT_DRAWINGS_POS_CURRENT@
method: min
['SK_ID_CURR'] 
INFO:logger:
num: AMT_DRAWINGS_CURRENT_diff_AMT_DRAWINGS_POS_CURRENT@
method: min
['SK_ID_CURR']
2018-08-08 08:36:23,410 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'AMT_DRAWINGS_CURRENT_diff_AMT_DRAWINGS_POS_CURRENT@_min@['SK_ID_CURR']'], dtype='object') 
2018-08-08 08:36:23,410 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'AMT_DRAWINGS_CURRENT_diff_AMT_DRAWINGS_POS_CURRENT@_min@['SK_ID_CURR']'], dtype='object') 
INFO:logger:
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'AMT_DRAWINGS_CURRENT_diff_AMT_DRAWINGS_POS_CURRENT@_min@

In [2]:
' 1段階の集計 value別'
prefix = 'ccb_'
df = ccb
level = unique_id
num_list = amt_list
num_list = cnt_list
num_list = month_list
method_list = ['sum', 'std', 'max', 'min']
method_list = ['sum', 'std', 'max']
method_list = ['sum', 'max', 'min']

select_category_value_agg(base=base, data=df, ignore_list=ignore_features,  level=level, cat_list=['NAME_CONTRACT_STATUS'], method_list=method_list, num_list=num_list, prefix=prefix)

In [38]:
pos = get_dummies(pos, cat_list)

In [42]:
' カテゴリ変数をダミー化して平均（出現率）を取得する '
prefix = 'pos_'
dummie_col = [col for col in pos.columns if col.count('CONTRACT_STATUS_')]

dummie_avg = pos[[unique_id] + dummie_col].groupby(unique_id).mean().reset_index()
for col in dummie_avg.columns:
    if col in ignore_features:
        continue
    dummie_avg.rename(columns={col:f'{col}@'}, inplace=True)
dummie_avg.head()

result = base.merge(dummie_avg, on=unique_id, how='left').fillna(0)
make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)

,SK_ID_CURR,NAME_CONTRACT_STATUS_Active@,NAME_CONTRACT_STATUS_Amortized debt@,NAME_CONTRACT_STATUS_Completed@,NAME_CONTRACT_STATUS_Demand@,NAME_CONTRACT_STATUS_Returned to the store@,NAME_CONTRACT_STATUS_Signed@
0,100001,0.777778,0.0,0.222222,0.0,0.0,0.000000
1,100002,1.000000,0.0,0.000000,0.0,0.0,0.000000
2,100003,0.928571,0.0,0.071429,0.0,0.0,0.000000
3,100004,0.750000,0.0,0.250000,0.0,0.0,0.000000
4,100005,0.818182,0.0,0.090909,0.0,0.0,0.090909


In [29]:
# ' 1段階の重み付き平均 '
# prefix = 'ccb_'
# # df = ccb
# df = amt_draw_sum
# # df = cnt_draw_sum
# level = unique_id
# # num_list = amt_list
# num_list = cnt_list
# num_list = [col for col in df.columns if col.count('per')]
# level = unique_id
# label = 'MONTHS_BALANCE'
# weight_list = [0.99, 0.97]

# for num in num_list:
#     if num==label:continue
#     for weight in weight_list:
#         wavg = exp_weight_avg(data=df, level=level, weight=weight, label=label, value=num)
#         result = base.merge(wavg.to_frame().reset_index(), on=unique_id, how='left').fillna(0)
#         make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)

In [1]:
' 2段階の集計 '
prefix = 'ccb_'
df = ccb
level = [unique_id, 'SK_ID_PREV']
num_list = amt_list
num_list = cnt_list
# num_list = month_list
method_list = ['sum', 'std', 'max', 'min']
method_list = ['sum', 'std', 'max']
# method_list = ['sum', 'max', 'min']

for method in method_list:
    method_list_2 = method_list.copy()
#     if method in ['sum', 'max', 'min']:
    if method in ['sum', 'max']:
        method_list_2.remove(method)
    for num in num_list:
        tmp = df.groupby(level)[num].agg({num:method}).reset_index().rename(columns={num:f'{num}_{method}@'})
    
        num_list_2 = [col for col in tmp.columns if col.count('@')]
        select_level_agg(base=base, data=tmp, ignore_list=ignore_features,  level=unique_id, method_list=method_list_2, num_list=num_list_2, prefix=prefix)

In [41]:
' 重み付き平均と集計 '
prefix = 'ccb_'
data = ccb
level = [unique_id, p_id]
label = 'MONTHS_BALANCE'
weight_list = [0.99, 0.97, 0.95]
num_list = amt_list
num_list = cnt_list

for num in num_list:
    if num==label:continue
    for weight in weight_list:
        wavg = exp_weight_avg(data=data, level=level, weight=weight, label=label, value=num)
        
        num_list = [wavg.name]
        wavg = wavg.to_frame().reset_index()
        select_level_agg(base=base, data=wavg, ignore_list=ignore_features, level=unique_id, method_list=method_list, num_list=num_list, prefix=prefix)

/mnt/c/Git/go/kaggle/kaggle_private/home-credit-default-risk/engineering/mv_wg_avg.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  null['weight'] = 0
2018-08-05 23:11:58,832 logger 222 [INFO]    [select_level_agg] 
num: CNT_DRAWINGS_ATM_CURRENT_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: sum
['SK_ID_CURR'] 
2018-08-05 23:11:58,832 logger 222 [INFO]    [select_level_agg] 
num: CNT_DRAWINGS_ATM_CURRENT_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: sum
['SK_ID_CURR'] 
../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})
2018-08-05 23:11:58,873 logger 227 [INFO]    [select_level_agg] 
base: 

2018-08-05 23:11:59,129 logger 30 [INFO]    [make_npy] 0.000000    24
0.121470     1
0.513587     1
0.033086     1
2.943089     1
0.118096     1
0.326540     1
1.247455     1
1.711563     1
0.131027     1
0.274231     1
0.185448     1
0.032238     1
0.839141     1
0.449176     1
1.074340     1
0.172859     1
0.066958     1
0.088579     1
1.788984     1
0.696465     1
0.127042     1
0.835133     1
0.002387     1
1.421050     1
4.806345     1
0.543182     1
0.452637     1
0.076988     1
0.088285     1
0.156069     1
0.045514     1
0.641083     1
0.141168     1
0.927286     1
0.164831     1
0.766655     1
0.032478     1
0.973094     1
0.038692     1
0.670202     1
0.821234     1
0.073898     1
0.087635     1
0.203555     1
0.440566     1
0.796234     1
0.310744     1
0.121366     1
0.247551     1
            ..
0.789691     1
0.123745     1
0.173343     1
0.918454     1
0.571739     1
0.790985     1
0.184986     1
0.595775     1
0.547358     1
0.875860     1
0.626460     1
0.078601     1


2018-08-05 23:12:04,239 logger 30 [INFO]    [make_npy] 0.000000    43526
0.112087      121
1.000000      102
0.098062       76
0.154238       75
0.130140       73
0.462713       70
0.077706       67
0.188006       63
0.086858       60
2.000000       59
0.323233       55
0.260280       54
0.001756       53
0.308475       53
0.224173       51
0.003511       49
0.376011       48
0.154238       48
0.238696       46
0.058159       45
0.564017       42
3.000000       42
0.390421       42
0.130140       40
0.053402       40
0.173717       39
0.196125       39
0.063661       39
0.155412       38
0.492386       38
0.260280       38
0.001810       34
0.294187       32
0.477393       30
0.005267       30
0.260575       28
0.646467       28
0.210274       28
0.448347       27
0.264305       27
0.070091       27
0.196125       27
0.070091       26
0.005617       26
0.004127       26
0.134165       26
0.336260       25
0.308475       25
0.098062       25
            ...  
0.037966        1
1.409972 

2018-08-05 23:12:04,546 logger 30 [INFO]    [make_npy] 0.000000    12162
0.112087      122
1.000000      102
0.098062       76
0.154238       75
0.130140       73
0.462713       71
0.077706       67
0.188006       64
0.086858       60
2.000000       60
0.323233       56
0.260280       55
0.308475       54
0.001756       53
0.224173       51
0.003511       49
0.376011       48
0.154238       48
0.238696       46
0.058159       45
0.564017       42
0.390421       42
3.000000       42
0.130140       40
0.053402       40
0.063661       39
0.196125       39
0.173717       39
0.492386       38
0.260280       38
0.155412       38
0.001810       34
0.294187       32
0.477393       31
0.005267       30
0.210274       28
0.260575       28
0.646467       28
0.264305       27
0.196125       27
0.448347       27
0.070091       27
0.004127       26
0.005617       26
0.070091       26
0.134165       26
0.098062       25
0.336260       25
0.308475       25
            ...  
0.008351        1
0.026961 

2018-08-05 23:12:09,678 logger 30 [INFO]    [make_npy] 0.000000    24
0.001829     2
0.008152     1
0.523510     1
0.845496     1
0.675952     1
0.344616     1
0.061494     1
2.561973     1
0.244257     1
1.382554     1
0.117196     1
0.031293     1
1.017262     1
0.145786     1
0.005307     1
0.835269     1
1.274414     1
0.032304     1
1.505857     1
1.023556     1
0.220514     1
0.166913     1
0.368231     1
0.479518     1
3.468924     1
0.666323     1
0.549368     1
0.121108     1
0.019828     1
1.523775     1
0.203970     1
0.007138     1
0.967377     1
0.251602     1
0.221726     1
0.083286     1
0.050819     1
0.071685     1
0.019266     1
0.159787     1
0.452483     1
0.060234     1
0.014795     1
0.009402     1
0.435362     1
0.811176     1
0.670618     1
0.111833     1
0.042113     1
            ..
0.150644     1
0.029307     1
0.143611     1
1.258493     1
2.705201     1
0.513444     1
0.503364     1
0.154998     1
1.536997     1
0.412681     1
1.314223     1
1.842291     1


...

2018-08-05 23:13:21,777 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: sum
['SK_ID_CURR'] 
2018-08-05 23:13:21,777 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: sum
['SK_ID_CURR'] 
2018-08-05 23:13:21,814 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']_sum@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:21,814 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']_sum@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:21,887 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:21,887 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:21,897 logger 30 [INFO]    [make_npy] 0.000

2018-08-05 23:13:22,082 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: max
['SK_ID_CURR'] 
2018-08-05 23:13:22,082 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: max
['SK_ID_CURR'] 
2018-08-05 23:13:22,113 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']_max@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:22,113 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.99_avg@['SK_ID_CURR', 'SK_ID_PREV']_max@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:22,190 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:22,190 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:22,199 logger 30 [INFO]    [make_npy] 0.000

2018-08-05 23:13:25,284 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.97_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: std
['SK_ID_CURR'] 
2018-08-05 23:13:25,284 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.97_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: std
['SK_ID_CURR'] 
2018-08-05 23:13:25,321 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.97_avg@['SK_ID_CURR', 'SK_ID_PREV']_std@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:25,321 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.97_avg@['SK_ID_CURR', 'SK_ID_PREV']_std@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:25,395 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:25,395 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:25,401 logger 30 [INFO]    [make_npy] 0.000

2018-08-05 23:13:28,321 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.95_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: sum
['SK_ID_CURR'] 
2018-08-05 23:13:28,321 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.95_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: sum
['SK_ID_CURR'] 
2018-08-05 23:13:28,359 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.95_avg@['SK_ID_CURR', 'SK_ID_PREV']_sum@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:28,359 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.95_avg@['SK_ID_CURR', 'SK_ID_PREV']_sum@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:28,428 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:28,428 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:28,437 logger 30 [INFO]    [make_npy] 0.000

2018-08-05 23:13:28,623 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.95_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: max
['SK_ID_CURR'] 
2018-08-05 23:13:28,623 logger 222 [INFO]    [select_level_agg] 
num: SK_DPD_DEF_wg0.95_avg@['SK_ID_CURR', 'SK_ID_PREV']
method: max
['SK_ID_CURR'] 
2018-08-05 23:13:28,655 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.95_avg@['SK_ID_CURR', 'SK_ID_PREV']_max@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:28,655 logger 227 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SK_DPD_DEF_wg0.95_avg@['SK_ID_CURR', 'SK_ID_PREV']_max@['SK_ID_CURR']'], dtype='object') 
2018-08-05 23:13:28,729 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:28,729 logger 230 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-08-05 23:13:28,739 logger 30 [INFO]    [make_npy] 0.000